<a href="https://colab.research.google.com/github/quantum-intelligence/computational-physics/blob/main/CP_Lecture9_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computational Physics
### Lecture 9
ODE and 1D quantum mechanics

NOTE: Need to save your work in Google Colab or your work may be lost

- For 'analytical' solution see  https://helentronica.com/2014/09/04/quantum-mechanics-with-the-python/
- For numerical solution see textbook


Solve the differential equation for the 1D square well
- use scipy.integrate.odeint() 


In [ ]:
from pylab import *
from scipy.integrate import odeint
from scipy.optimize import brentq
import numpy as np

Use scipy for physical constants

In [ ]:
from scipy.constants import *

print("hbar", hbar)
el = 1.602176634e-19
hbar_c = hbar/e
print("hbar_c", hbar_c)
print("m_e",m_e,"m_p",m_p)
c_fm = c*1E12
mc2 = (m_e*c**2)/e
print("mc2","{:e}".format(mc2/e))
print("hbar*c","{:e}".format(hbar_c*c/1E-10))
prefactor = ((hbar_c*c/1E-10)**2/(2*mc2))**(-1)
print("check prefactor:", prefactor, "eV^-1 Angstrom^-2")


hbar 1.0545718176461565e-34
hbar_c 6.582119569509067e-16
m_e 9.1093837015e-31 m_p 1.67262192369e-27
mc2 3.189405e+24
hbar*c 1.973270e+03
check prefactor: 0.2624684236075175 eV^-1 Angstrom^-2


## Numerical solution to 1D QW
- Inspect helper functions and main functinos below
- Adjust parameters as needed
- Plot $psi(x)$ and V(x) to check results
- Compare with analytical results to verify your work
  - ('analytical' check needs to modified )

In [ ]:
def SE(psi, x, E):
    """
    Returns derivatives for the 1D schrodinger eq.
    Requires global value E to be set somewhere. State0 is
    first derivative of the wave function psi, and state1 is
    its second derivative.
    """
    f1 = psi[1]
    f2 = 0.2624*(V(x) - E)*psi[0]
    return [f1, f2]


In [ ]:
def V(x):
    """
    Potential function in the finite square well. Width is L and value is global variable Vo
    """
    L = 4
    V0 = -20
    if abs(x) < L:
        return V0
    else:
        return 0

In [ ]:
def wf_calc(SE, psi0, x, E):
    """
    Calculates wave function psi for the given value
    of energy E and returns value at point b
    """
    #print('psi0',psi0)
    psi = odeint(SE, psi0, x, args=(E,))
    return psi

In [ ]:
def log_diff(E,h):
  y = zeros((2),float)
  i_match = n_steps//3
  nL = i_match + 1
  y[0] = 1.E-25
  y[1] = y[0]*sqrt(E*0.2624)
  ix = np.arange(0, nL+1)
  x = h*(ix - n_steps/2)
  psi = wf_calc(SE, y, x, E)
  left = psi[-1,1]/psi[-1,0]
  y[0] = 1.E-25
  y[1] = -y[0]*sqrt(E*0.2624)
  ix = np.arange(n_steps, nL+1,-1)
  x = h*(ix + 1- n_steps/2)
  psi = wf_calc(SE, y, x, E)
  right = psi[-1,1]/psi[-1,0]
  return (left - right)/(left + right)


In [ ]:
#initialize parameters
eps = 1E-3
n_steps = 501
E = 11.537 #-17
h = 0.016  #well width = (n_steps-1)*h --> 8Angstroms
count_max = 3000
Emax = 1.1*E
Emin = E/1.1
print("Emin: ", Emin,"Emax: ", Emax)

Emin:  10.488181818181818 Emax:  12.690700000000001


In [ ]:
#main script
for count in range(0, count_max + 1):
  E = (Emax + Emin)/2
  Diff = log_diff(E, h)
  if (log_diff(Emax,h)*Diff > 0): #bisection algorithm
    Emax = E
  else:
    Emin = E
  if (abs(Diff) < eps):
    break

print("final eigenvalues E = ", E)
print("iterations, max = ", count)

final eigenvalues E =  10.488181818181818
iterations, max =  3000


Energies from an analytical estimate: 
- need to modify with appropriate paramters/units


In [ ]:
def find_analytic_energies(en):
    """
    Calculates Energy values for the finite square well using analytical
    model (Griffiths, Introduction to Quantum Mechanics, 1st edition, page 62.)
    """
    a = 8 #length of quantum well

    #z = sqrt(2*m_e/el*en)/hbar_c/1E-10
    #z0 = a/hbar_c*sqrt((1/2)*m_e*Vo)/1E-10
    z = sqrt(2*en)
    z0 = sqrt(2*Vo)

    z_zeroes = []

    #f_sym = lambda z: tan(z*a/3)-sqrt((z0/z)**2-1)     # symmetrical case
    #f_asym = lambda z: -1/tan(z*a/2)-sqrt((z0/z)**2-1) # antisymmetrical case
    f_sym = lambda z: tan(z/2)-sqrt((z0/z)**2-1)      # symmetrical case
    f_asym = lambda z: -1/tan(z/2)-sqrt((z0/z)**2-1)  # antisymmetrical case

    # first find the zeroes for the symmetrical case
    s = sign(f_sym(z))
    for i in range(len(s)-1):   # find zeroes of this crazy function
       if s[i]+s[i+1] == 0:
           zero = brentq(f_sym, z[i], z[i+1])
           z_zeroes.append(zero)
    print("Energies from the analyitical model are: ")
    print("Symmetrical case)")
    for i in range(0, len(z_zeroes),2):   # discard z=(2n-1)pi/2 solutions cause that's where tan(z) is discontinous
        print("%.4f" %(z_zeroes[i]**2/2))
    # Now for the asymmetrical
    z_zeroes = []
    s = sign(f_asym(z))
    for i in range(len(s)-1):   # find zeroes of this crazy function
       if s[i]+s[i+1] == 0:
           zero = brentq(f_asym, z[i], z[i+1])
           z_zeroes.append(zero)
    print("(Antisymmetrical case)")
    for i in range(0, len(z_zeroes),2):   # discard z=npi solutions cause that's where ctg(z) is discontinous
        #print("%8.4f" % (z_zeroes[i]**2/2) )
        print( (z_zeroes[i]**2/2) )


In [ ]:
Vo = 20
en = linspace(0.001, Vo, 1000) 
find_analytic_energies(en)


Energies from the analyitical model are: 
Symmetrical case)
2.8144
19.9920
(Antisymmetrical case)
10.751611827340433


Question:
How else can we check our result? (Hint: use limits)